In [1]:
!pip install gradio pandas numpy matplotlib scikit-learn statsmodels keras prophet


In [2]:
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from keras.models import Sequential
from keras.layers import LSTM, Dense
from prophet import Prophet
import time
import traceback
import sklearn
print(sklearn.__version__)

1.6.0


In [3]:
# ARIMA Forecast
def forecast_arima(data, periods, p=1, d=1, q=1):
    try:
        model = ARIMA(data, order=(p, d, q))
        fitted_model = model.fit()
        forecast = fitted_model.forecast(steps=periods)
        return forecast
    except Exception as e:
        print(f"ARIMA Error: {e}")
        return np.zeros(periods)

# LSTM Forecast
def forecast_lstm(data, periods, neurons=50, epochs=10):
    try:
        data = data.values.reshape(-1, 1)
        model = Sequential()
        model.add(LSTM(neurons, activation='relu', input_shape=(1, 1)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
        model.fit(data[:-periods], data[:-periods], epochs=epochs, verbose=0)
        forecast = model.predict(data[-periods:].reshape(periods, 1, 1))
        return forecast.flatten()
    except Exception as e:
        print(f"LSTM Error: {e}")
        return np.zeros(periods)

# Prophet Forecast
def forecast_prophet(data, periods, seasonality="auto"):
    try:
        df = pd.DataFrame({'ds': data.index, 'y': data.values})
        model = Prophet(yearly_seasonality=seasonality)
        model.fit(df)
        future = model.make_future_dataframe(periods=periods)
        forecast = model.predict(future)
        return forecast['yhat'].values[-periods:]
    except Exception as e:
        print(f"Prophet Error: {e}")
        return np.zeros(periods)

# Hybrid Model Forecast
def forecast_hybrid(data, periods, arima_params=(1, 1, 1), lstm_params=(50, 10), seasonality="auto"):
    try:
        arima_forecast = forecast_arima(data, periods, *arima_params)
        lstm_forecast = forecast_lstm(data, periods, neurons=lstm_params[0], epochs=lstm_params[1])
        prophet_forecast = forecast_prophet(data, periods, seasonality=seasonality)
        return (arima_forecast + lstm_forecast + prophet_forecast) / 3
    except Exception as e:
        print(f"Hybrid Error: {e}")
        return np.zeros(periods)


In [4]:
# Validate and parse CSV
def validate_csv(data):
    try:
        if data.shape[1] < 2:
            return False, "CSV must have at least two columns: Date and Value."
        
        # Attempt to parse the date column with the correct format
        pd.to_datetime(data.iloc[:, 0], dayfirst=True)  # This ensures 'day-first' format
        if not pd.api.types.is_numeric_dtype(data.iloc[:, 1]):
            return False, "Second column must contain numeric values."
        
        return True, None
    except Exception as e:
        return False, f"CSV Validation Error: {e}"

In [5]:
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-10))) * 100

def smape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    return np.mean(np.abs(y_true - y_pred) / (denominator + 1e-10)) * 100

In [6]:
def run_forecasting(domain, file, model_types, time_frequency, graph_type, p, d, q, neurons, epochs, seasonality):
    try:
        # Read the CSV file
        data = pd.read_csv(file.name)
        
        # Check for required columns
        if 'Date' not in data.columns or 'Value' not in data.columns:
            return "CSV file must contain 'Date' and 'Value' columns.", None, None, None, None
        
        # Validate date column and handle invalid date format
        data.columns = ["Date", "Value"]
        data["Date"] = pd.to_datetime(data["Date"], errors='coerce')
        if data["Date"].isna().any():
            return "Some dates are invalid. Please check the 'Date' column.", None, None, None, None
        
        data.set_index("Date", inplace=True)

        # Period mapping based on frequency
        periods_map = {
            "Monthly": (12, 'M'), "Yearly": (12, 'M'),
            "Quarterly": (4, 'Q'), "Daily": (30, 'D'),
            "Hourly": (24, 'H'), "Minute-wise": (60, 'T'),
            "Second-wise": (60, 'S'), "Weekly": (52, 'W')
        }

        forecast_periods, freq = periods_map.get(time_frequency, (30, 'D'))

        # Dictionary for storing results
        times = {}
        forecasts = {}
        metrics = {}

        # Loop through each model type
        for model_type in model_types:
            start_time = time.time()

            # Forecasting based on model type
            if model_type == 'ARIMA':
                forecast = forecast_arima(data['Value'], forecast_periods, p, d, q)
            elif model_type == 'LSTM':
                forecast = forecast_lstm(data['Value'], forecast_periods, neurons=neurons, epochs=epochs)
            elif model_type == 'PROPHET':
                forecast = forecast_prophet(data['Value'], forecast_periods, seasonality=seasonality)
            elif model_type == 'HYBRID':
                forecast = forecast_hybrid(data['Value'], forecast_periods, (p, d, q), (neurons, epochs), seasonality)

            # Calculate performance metrics
            true_values = data['Value'][-forecast_periods:].values if len(data) > forecast_periods else np.zeros(forecast_periods)
            mae = mean_absolute_error(true_values, forecast)
            rmse = np.sqrt(mean_squared_error(true_values, forecast))
            mape_value = mape(true_values, forecast)
            smape_value = smape(true_values, forecast)

            forecasts[model_type] = forecast
            metrics[model_type] = {"MAE": mae, "RMSE": rmse, "MAPE": mape_value, "SMAPE": smape_value}
            times[model_type] = time.time() - start_time

        # Create a metrics DataFrame
        metrics_df = pd.DataFrame({
            "Model": list(metrics.keys()),
            "MAE": [metrics[m]["MAE"] for m in metrics],
            "RMSE": [metrics[m]["RMSE"] for m in metrics],
            "MAPE": [metrics[m]["MAPE"] for m in metrics],
            "SMAPE": [metrics[m]["SMAPE"] for m in metrics]
        })

        # Identify the best model based on MAE
        best_model = min(metrics, key=lambda x: metrics[x]["MAE"])

        # Generate forecast dates
        forecast_dates = pd.date_range(data.index[-1], periods=forecast_periods, freq=freq)

        # Plot the forecast results
        plt.figure(figsize=(12, 6))
        if graph_type == "Line":
            plt.plot(data.index, data['Value'], label='Historical Data', color='blue', marker='o')
            for model_type, forecast in forecasts.items():
                plt.plot(forecast_dates, forecast, label=f'{model_type} Forecast')
        elif graph_type == "Bar":
            for i, (model_type, forecast) in enumerate(forecasts.items()):
                plt.bar(forecast_dates + pd.Timedelta(days=i), forecast, width=0.2, label=f'{model_type} Forecast')
        elif graph_type == "Histogram":
            plt.hist(data['Value'], bins=20, alpha=0.7, label='Historical Data')
            for model_type, forecast in forecasts.items():
                plt.hist(forecast, bins=20, alpha=0.7, label=f'{model_type} Forecast')
        elif graph_type == "Scatter":
            plt.scatter(data.index, data['Value'], label='Historical Data', color='blue')
            for model_type, forecast in forecasts.items():
                plt.scatter(forecast_dates, forecast, label=f'{model_type} Forecast')

        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.title(f"{domain} Forecast ({time_frequency})")
        plt.legend()
        plt.grid(True)
        
        # Save plot with a dynamic filename
        plot_filename = f"forecast_plot_{time.time()}.png"
        plt.savefig(plot_filename)
        plt.close()

        # Prepare forecast table
        forecast_table = pd.DataFrame({
            "Date": forecast_dates,
            **{model_type: forecasts[model_type] for model_type in forecasts}
        })

        # Time taken DataFrame
        time_df = pd.DataFrame(times.items(), columns=['Model', 'Time Taken (s)'])
        comment = f"The best-performing model based on MAE is: {best_model}."

        return comment, plot_filename, metrics_df, time_df, forecast_table

    except Exception as e:
        print(traceback.format_exc())
        return f"An error occurred: {str(e)}", None, None, None, None

In [7]:
interface = gr.Interface(
    fn=run_forecasting,
    inputs=[
        gr.Dropdown(choices=["Finance", "Weather", "Healthcare", "Retail Sales", "Traffic", "Crime", "Energy"], label="Select Domain"),
        gr.File(label="Upload CSV"),
        gr.CheckboxGroup(choices=["ARIMA", "LSTM", "PROPHET", "HYBRID"], label="Select Forecasting Models"),
        gr.Radio(choices=["Monthly", "Yearly", "Quarterly", "Daily", "Hourly", "Minute-wise", "Second-wise", "Weekly"], label="Select Time Frequency"),
        gr.Radio(choices=["Line", "Bar", "Histogram", "Scatter"], label="Select Graph Type"),
        gr.Slider(0, 5, value=1, step=1, label="ARIMA (p)"),
        gr.Slider(0, 2, value=1, step=1, label="ARIMA (d)"),
        gr.Slider(0, 5, value=1, step=1, label="ARIMA (q)"),
        gr.Slider(10, 100, value=50, step=10, label="LSTM Neurons"),
        gr.Slider(1, 50, value=10, step=1, label="LSTM Epochs"),
        gr.Dropdown(choices=["auto", "additive", "multiplicative"], label="Prophet Seasonality")
    ],
    outputs=[
        gr.Text(label="Best Model Comment"),
        gr.Image(label="Forecast Graph"),
        gr.Dataframe(label="Metrics Table"),
        gr.Dataframe(label="Time Taken Table"),
        gr.Dataframe(label="Forecast Data Table")
    ],
    title="Time Series Forecasting App",
    description="Upload your data to compare forecasting models and visualize results."
)

interface.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
